In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
import httplib
from pykafka import KafkaClient

conf = SparkConf().setAppName("Sales Order Profile Update").setMaster("spark://server1:7077")
conf.set("spark.cores.max", 3)
conf.set("spark.cassandra.connection.host", "server4")
#conf.set("spark.executor.extraClassPath", "/root/spark-cassandra-connector-assembly-1.5.0-M1-SNAPSHOT.jar")
#conf.set("spark.executor.extraClassPath", "/root/spark-streaming-kafka-assembly_2.10-1.5.0.jar")
#conf.set("spark.executor.extraClassPath", "/root/kafka_2.10-0.8.2.1/libs/kafka_2.10-0.8.2.1.jar")
#conf.set("spark.executor.extraClassPath", "/root/kafka_2.10-0.8.2.1/libs/zkclient-0.3.jar")


sc = SparkContext(conf=conf)

# Create a local StreamingContext with two working thread and batch interval of 1 second
# need change local[2] to spark://server1:7077
#sc = SparkContext("spark://server1:7077", "profile_update")
#sc = SparkContext("local[2]", "profile_update")
streamingContext = StreamingContext(sc, 30)

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

class SimpleClient(object):
    session = None

    def connect(self, nodes):
        cluster = Cluster(nodes)
        self.session = cluster.connect()
        #keyspace is test
        self.session.execute("use test")
        
    def close(self):
        self.session.cluster.shutdown()
    
    def execute(self, statement, parameters=[]):
        preparedStatement = self.session.prepare(statement)
        return self.session.execute(preparedStatement, parameters)

In [ ]:
#change localhost:2181 to server7 or zookeeper on server1?
stream = KafkaUtils.createStream(streamingContext, 'server1.bigdata.ibm.com:2181', "profile-update", {'profile' :1})


def processEvent(client, profile):
    if(profile.has_key('Score')):
        #print '--------update profile----------'
        #print profile
        client.execute("""update customer_product 
            set LTDSalesAmt = ?, LTDSalesQty = ?, LTDSalesCount = ?, AvgSalesAmt = ?, AvgSalesQty = ?
            where customer = ? and product = ? and date = ?""",
                    [profile['LTDSalesAmt'], profile['LTDSalesQty'], profile['LTDSalesCount'], 
                    profile['AvgSalesAmt'], profile['AvgSalesQty'], 
                    profile['customer'], profile['product'], profile['date']])
    
        print '######## update profile:' + str(profile)
        return profile
    
def processPartition(iter):
    try:
        #cassandra client
        client = SimpleClient()
        #change hostname to server4 which is the cassandra 
        client.connect(['server4.bigdata.ibm.com','server5.bigdata.ibm.com','server6.bigdata.ibm.com'])
        #client.connect(['sandbox.hortonworks.com'])
        #print 'connected'
        #kafka producer
        #change hostname to server1, which is zookeeper? or server7 which is kafka
        #kafkaClient = KafkaClient(hosts="server7.bigdata.ibm.com:9092")
        #kafkaClient = KafkaClient(hosts="sandbox.hortonworks.com:6667")
        #topic = kafkaClient.topics['profile']
        #producer = topic.get_producer()
        for record in iter:
            #profile = processEvent(client, producer, json.loads(record[1]))
            profile = processEvent(client, json.loads(record[1]))
    finally:
        #print 'disconnected'
        client.close()
        

stream.foreachRDD(lambda rdd: rdd.foreachPartition(processPartition))

In [ ]:
streamingContext.start()             # Start the computation
streamingContext.awaitTermination()  # Wait for the computation to terminate